# Indexing Crosswords

- After running on sitemaps 9 and 10: N indexed, N errored (2959 total)

In [1]:
%load_ext nb_black

import traceback
import datetime
import os
import re
import time
import random
import json
from collections import defaultdict

import requests
import bs4
import numpy as np
import pandas as pd

import ipdb

from cryptic_info.parse import try_parse

<IPython.core.display.Javascript object>

In [2]:
from cryptic_info.tables import *
from cryptic_info.lists import *
from cryptic_info.utils import extract_puzzle_url

<IPython.core.display.Javascript object>

In [3]:
headers = {
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64)",
    "Accept-Encoding": "gzip",
}

<IPython.core.display.Javascript object>

## Getting URLs

## Testing out

In [8]:
# Tables - type 1
# source_url = (
# "https://www.fifteensquared.net/2021/05/20/financial-times-16790-by-leonidas/"
# "https://www.fifteensquared.net/2021/05/21/financial-times-16791-by-buccaneer/"
# "https://www.fifteensquared.net/2021/05/21/independent-10797-by-phi/"
# "https://www.fifteensquared.net/2021/05/23/azed-no-2553-plain/"
# "https://www.fifteensquared.net/2021/05/23/everyman-3892/"
# "https://www.fifteensquared.net/2021/05/30/azed-no-2554-plain/"
# )

# Tables - type 2
# source_url = "https://www.fifteensquared.net/2021/05/17/guardian-28447-anto/"

# Table - type 3
# source_url = "http://www.fifteensquared.net/2021/05/22/independent-10798-by-alchemi-saturday-puzzle-22-may-2021/"
# source_url = "http://www.fifteensquared.net/2021/05/24/cyclops-702-gigantic-hiccup/"

# Table - type 4
source_url = "https://www.fifteensquared.net/2020/09/06/azed-2516/"
source_url = "https://www.fifteensquared.net/2020/09/02/independent-10574-eccles/"
source_url = "https://www.fifteensquared.net/2020/09/08/independent-10579-kairos/"

# List - type 1
# source_url = (
# "https://www.fifteensquared.net/2021/05/22/guardian-saturday-puzzle-28446-tramp/"
# "https://www.fifteensquared.net/2021/05/23/independent-on-sunday-1630-by-raich/"
# "https://www.fifteensquared.net/2021/05/19/guardian-28449-pasquale/"
# "https://www.fifteensquared.net/2021/05/17/guardian-quiptic-1122-hectence/"
# "https://www.fifteensquared.net/2021/05/16/independent-on-sunday-1629-hoskins/"
# )

# List - type 2
# source_url = (
# "https://www.fifteensquared.net/2021/05/20/independent-10796-by-tees/"
# "https://www.fifteensquared.net/2021/05/17/financial-times-16787-by-peto/"
# "https://www.fifteensquared.net/2021/06/02/guardian-28461-imogen/"
# "https://www.fifteensquared.net/2021/06/01/independent-10806-by-kairos/"
# "http://www.fifteensquared.net/2021/05/15/independent-10792-by-monk/"
# )

# List - type 3
# source_url = "https://www.fifteensquared.net/2021/06/01/financial-times-16800-chalmie/"
# source_url = "https://www.fifteensquared.net/2021/05/21/guardian-cryptic-28451-puck/"
# source_url = "https://www.fifteensquared.net/2021/05/24/guardian-quiptic-1123-matilda/"

# Hihoba does hard themed puzzles, and formats their posts fairly inconsistently, depending on the theme
# https://www.fifteensquared.net/author/hihoba/
# source_url = "https://www.fifteensquared.net/2021/05/18/inquisitor-1698-spooky-manifestations-by-kruger/"

# RatkojaRiku does not include clues with their posts.
# https://www.fifteensquared.net/author/ratkojariku/
# source_url = "https://www.fifteensquared.net/2021/05/13/independent-10790-serpent/"

# FIXME: why does the extract_definitions fail? Not urgent
# source_url = "https://www.fifteensquared.net/2021/05/16/everyman-3891/"

# TODO
# source_url = "https://www.fifteensquared.net/2020/09/03/independent-10575-dalibor/"
# source_url = "https://www.fifteensquared.net/2020/09/05/guardian-prize-28225-by-maskarade/"
# source_url = "https://www.fifteensquared.net/2020/09/09/independent-10580-tees/"

response = requests.get(source_url, headers=headers)

<IPython.core.display.Javascript object>

In [9]:
data = try_parse(response, source_url)
data

Parsing using parse_table_type_4


,ClueNumber,Clue,Annotation,Answer,Definition,PuzzleURL,SourceURL
0,1a,Reportedly remained sober (5),STAID (sounds like [reportedly] STAYED [remain...,STAID,sober,None,https://www.fifteensquared.net/2020/09/08/inde...
1,4a,Copper’s covering evidence (5),P (penny; copper [coin]) + ROOF (covering) P R...,PROOF,evidence,None,https://www.fifteensquared.net/2020/09/08/inde...
2,9a,Time to greet prince going with crusader (9),WEEP (cry tears; greet [Scottish term for cry ...,WEEKNIGHT,Time,None,https://www.fifteensquared.net/2020/09/08/inde...
3,10a,Trouble with one consuming old mayonnaise (5),(AIL [trouble] + I [Roman numeral for one]) co...,AIOLI,nan,None,https://www.fifteensquared.net/2020/09/08/inde...
4,11a,Slow-witted Hobbits regularly ignored exercise...,OBT (letters remaining in HOBBITS after letter...,OBTUSE,Slow-witted,None,https://www.fifteensquared.net/2020/09/08/inde...
5,13a,"Society priest maybe pursuing Esther (4,4)",BOOK (Esther is a BOOK in the Old Testament of...,BOOK CLUB,Society,None,https://www.fifteensquared.net/2020/09/08/inde...
6,14a,Old chess player dumps Charlie for Rod (6),"FISCHER (reference Bobby FISCHER [1943-2008], ...",FISHER,Rod,None,https://www.fifteensquared.net/2020/09/08/inde...
7,16a,Aerobic exercises incorporating Latin dance mo...,Anagram of (exercises) AEROBIC containing (inc...,CABRIOLE,dance movement,None,https://www.fifteensquared.net/2020/09/08/inde...
8,19a,Old children’s programme‘s in top 100 (8),TRUMP (top) + TON (one hundred; 100) TRUMP TON...,TRUMPTON,Old children’s programme,None,https://www.fifteensquared.net/2020/09/08/inde...
9,20a,Plant working for china maker (6),MINT (herb; plant) + ON (working) MINT ON (an ...,MINTON,china maker,None,https://www.fifteensquared.net/2020/09/08/inde...


<IPython.core.display.Javascript object>